### **PRE PROCESSAMENTO**

In [ ]:
import pandas as pd 
df = pd.DataFrame()

In [ ]:
def remove_outlier(df):
    Q1 = df.quatile(0.25)
    Q3 = df.quatile(0.75)
    IQ = Q3 - Q1
    LI = df < (Q1 - 1.5*IQ)
    LS = df > (Q3 + 1.5*IQ)

    df_final = df[~(LI | LS)]
    return df_final

In [ ]:
# Removendo Outliers de um dataFrame inteiro
df_normal = remove_outlier(df)

# Se quiser saber a diferença entre um DataFrame e outro:
    # Retorna apenas os indices das linhas que contem os outiliers
indice_dif = df.index.difference(df_normal.index)

In [ ]:
for i in range(indice_dif, 1):
    df.loc[df_dif[i], 'coluna_que_quero'] = media_de_algum_valor

### **VERIFICANDO QUANTIDADE DE CLUSTERs**

In [ ]:
# Calculando a quantidade de Clusters com base na Soma dos Quadrados (Cotovelo)
def qtd_cluster(df):
    wcss = []
    for i in range(1, 10):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(df)  
        wcss.append(kmeans.inertia_)
    return wcss



# Função para calcular a quantidade ideal de Clusters para minha base
# Soma dos Quadrados é o Vetor anterior 
def numero_otimo_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss) - 1]

    distancia = []
    for i in range(len(wcss)):
        x0 = i + 2
        y0 = wcss[i]
        numerador = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominador = math.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)
        distancia.append(numerador / denominador)
    
    return distancia.index(max(distancia)) + 2

### **CRIANDO KLUSTER**

In [ ]:
kmenas= KMeans(n_clusters=4)
df_usuario['RecenciaCluster'] = kmenas.fit_predict(df_recencia)
df_usuario.head()

### **ORDENAÇÃO DO CLUSTER**

In [ ]:
# Cuidar a ordenação para ver se ela veio de forma correta
def ordena_cluster(nome_cluster, alvo, df, ordem):
    agrupado_por_cluster = df.groupby(nome_cluster)['Recencia'].mean().reset_index()
    agrupado_por_cluster_ordenado = agrupado_por_cluster.sort_values(by='Recencia', ascending=ordem).reset_index(drop=True)
    agrupado_por_cluster_ordenado['index'] = agrupado_por_cluster_ordenado.index
    juntando_cluster = pd.merge(df, agrupado_por_cluster_ordenado[[nome_cluster, 'index']], on = nome_cluster)
    removendo_dados = juntando_cluster.drop(nome_cluster, axis=1)
    df_final = removendo_dados.rename(columns={'index':nome_cluster})
    
    return df_final
    

### **VISUALIZAÇÃO DOS CLUSTERS**

In [ ]:
# Cuidar a ordenação para ver se ela veio de forma correta
def plot_segmento(x, y, data):
    sns.set()  # Configura o estilo do gráfico
    plt.figure(figsize=(7, 5))  # Define o tamanho da figura
    sns.scatterplot(x=x, y=y, hue='Segmento', data=data, 
                    size='Segmento', sizes=(50, 150), 
                    size_order=['Premium', 'Master', 'Busines', 'Inativo'], 
                    palette='crest')  # Define a paleta de cores

    plt.show()  # Exibe o gráfico
    